In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

actions = ['cold', 'hot']
seq_length = 30

# 저장된 모델 로드
model = load_model('/Users/ihogyun/models/final_model.keras')

# MediaPipe Pose 모델 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = pose.process(img_rgb)
    img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    if result.pose_landmarks is not None:
        landmarks = result.pose_landmarks.landmark
        
        # 랜드마크 포인트 저장 (어깨, 팔꿈치, 손목 등)
        joint = np.zeros((len(mp_pose.PoseLandmark), 4))
        for i, lm in enumerate(landmarks):
            joint[i] = [lm.x, lm.y, lm.z, lm.visibility]

        # 상반신의 주요 관절 포인트
        left_shoulder = joint[mp_pose.PoseLandmark.LEFT_SHOULDER.value][:3]
        left_elbow = joint[mp_pose.PoseLandmark.LEFT_ELBOW.value][:3]
        left_wrist = joint[mp_pose.PoseLandmark.LEFT_WRIST.value][:3]
        right_shoulder = joint[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][:3]
        right_elbow = joint[mp_pose.PoseLandmark.RIGHT_ELBOW.value][:3]
        right_wrist = joint[mp_pose.PoseLandmark.RIGHT_WRIST.value][:3]

        # 각도 계산
        def calculate_angle(v1, v2):
            v1 = v1 / np.linalg.norm(v1)  # Normalize
            v2 = v2 / np.linalg.norm(v2)  # Normalize
            angle = np.arccos(np.dot(v1, v2))
            return np.degrees(angle)

        left_arm_angle = calculate_angle(left_elbow - left_shoulder, left_wrist - left_elbow)
        right_arm_angle = calculate_angle(right_elbow - right_shoulder, right_wrist - right_elbow)

        # 데이터 저장
        angle_data = np.array([left_arm_angle, right_arm_angle], dtype=np.float32)
        d = np.concatenate([joint.flatten(), angle_data])

        seq.append(d)

        mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        if len(seq) < seq_length:
            continue

        input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

        y_pred = model.predict(input_data).squeeze()

        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]

        if conf < 0.9:
            continue

        action = actions[i_pred]
        action_seq.append(action)

        if len(action_seq) < 3:
            continue

        this_action = '?'
        if action_seq[-1] == action_seq[-2] == action_seq[-3]:
            this_action = action

        cv2.putText(img, f'{this_action.upper()}', org=(int(landmarks[0].x * img.shape[1]), int(landmarks[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




I0000 00:00:1723289390.377836 1218023 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1723289390.476173 1218593 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723289390.484882 1218593 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/ihogyun/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━